In [2]:
import pandas as pd
import glob
import numpy as np
import os

# --- 1️⃣ Load and combine all team-season files ---
all_files = glob.glob("team_game_data/*.parquet")
df = pd.concat([pd.read_parquet(f) for f in all_files], ignore_index=True)

In [4]:
df.head()

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612752,0022301190,"APR 14, 2024",NYK vs. CHI,W,50,32,0.610,265,46,...,0.762,16,37,53,27,7,6,21,17,120
1,1610612752,0022301175,"APR 12, 2024",NYK vs. BKN,W,49,32,0.605,240,38,...,0.840,8,23,31,21,10,3,5,15,111
2,1610612752,0022301167,"APR 11, 2024",NYK @ BOS,W,48,32,0.600,240,49,...,0.600,17,35,52,24,7,4,12,12,118
3,1610612752,0022301149,"APR 09, 2024",NYK @ CHI,W,47,32,0.595,240,46,...,0.950,6,25,31,31,6,4,10,15,128
4,1610612752,0022301139,"APR 07, 2024",NYK @ MIL,W,46,32,0.590,240,45,...,0.818,13,28,41,25,9,6,5,22,122


In [5]:
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format='%b %d, %Y')
df = df.sort_values(['Team_ID', 'GAME_DATE']).reset_index(drop=True)
df['TARGET_WL'] = df['WL'].map({'W': 1, 'L': 0})
df = df.drop(columns=['WL', 'W_PCT', 'MATCHUP'])

In [12]:
df.head()

,Team_ID,Game_ID,GAME_DATE,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,TARGET_WL
0,1610612737,0022100014,2021-10-21,W,240,45,94,0.479,15,35,...,6,49,55,31,8,9,13,16,113,1
1,1610612737,0022100027,2021-10-23,L,240,38,99,0.384,10,34,...,17,37,54,20,5,3,9,23,95,0
2,1610612737,0022100043,2021-10-25,W,240,46,90,0.511,12,32,...,10,39,49,24,11,3,13,19,122,1
3,1610612737,0022100059,2021-10-27,W,240,40,96,0.417,8,30,...,21,34,55,21,4,4,11,14,102,1
4,1610612737,0022100066,2021-10-28,L,240,48,88,0.545,6,21,...,6,37,43,26,4,3,13,16,111,0


In [10]:
target = 'TARGET_WL'
features = [col for col in df.columns if col not in ['TEAM_ID', 'Game_ID', 'GAME_DATE', 'OPP_TEAM', target]]

df_model = df.dropna(subset=features + [target])

# Split chronologically (e.g., 80% train, 20% test)
split_idx = int(len(df_model) * 0.8)
train_df = df_model.iloc[:split_idx]
test_df = df_model.iloc[split_idx:]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

ValueError: could not convert string to float: 'W'